In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from numpy import random
from tqdm import tqdm
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn import pipeline
from sklearn.feature_selection import SelectKBest, f_regression, f_classif, chi2, SelectFromModel
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.cluster import KMeans
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from scipy.stats import pearsonr
from sklearn.metrics import confusion_matrix, classification_report, silhouette_score, f1_score, precision_recall_curve, plot_precision_recall_curve, average_precision_score, auc
from sklearn.svm import SVR, LinearSVR, SVC, LinearSVC
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder
from sklearn.kernel_approximation import Nystroem
from sklearn.linear_model import SGDClassifier, SGDRegressor
from sklearn.neural_network import MLPClassifier, MLPRegressor
from scipy import stats
from keras.layers import Dense
from keras.models import Sequential
from sklearn.metrics import plot_confusion_matrix

#Population generation for 2001 and 2011

In [ ]:
year = '2001'

In [ ]:
vill_csv = pd.read_csv('/content/drive/MyDrive/Scripts_and_Data_Transfer/5.ADI_Analysis/2011_Original(Ground_Truth)_ADI_without_outlier/vill.csv')
df_result = vill_csv.copy()
print(len(vill_csv))
# vill_csv.head()

vill_csv['Village_ID'] = vill_csv['Town/Village']
vill_csv.set_index('Village_ID', inplace = True)
vill_csv = vill_csv[vill_csv['Level'] != 'WARD']
vill_csv = vill_csv[np.logical_not(vill_csv.index.duplicated())]
print(len(vill_csv))
# vill_csv.head()

IN11 = pickle.load(open('/content/drive/MyDrive/Scripts_and_Data_Transfer/4.Training_CNN_(Arch_3)/Data/2nd_Level_Regression_Inp/household_input_2011_arch3_viirs_popAdd.pickle', 'rb'))

if year == '2001':
  mapping = pickle.load(open('/content/drive/MyDrive/Scripts_and_Data_Transfer/4.Training_CNN_(Arch_3)/Data/village_code_mapping/2011_2001_corr.pickle', 'rb'))
  IN11 = IN11.copy().rename(index=mapping)
  IN11.index = pd.to_numeric(IN11.index, errors='coerce').astype('Int64')
  # IN11['Village_ID'] = pd.to_numeric(IN11['Village_ID'], errors='coerce').astype('Int64')
  IN11 = IN11[IN11.index.notnull()]

  IN11 = IN11.dropna()

print(len(IN11))
# IN11.head()

vill_csv = vill_csv.filter(items = IN11.index, axis = 0)
print(len(vill_csv))
vill_csv.head()

690391
605412
340949
6677


,Unnamed: 0,Unnamed: 0.1,State,District,Subdistt,Town/Village,Ward,EB,Level,Name,...,Village_HHD_Cluster_MSW,District_HHD_Cluster_MSW,Village_HHD_Cluster_CHH,District_HHD_Cluster_CHH,Village_HHD_Cluster_FC,District_HHD_Cluster_FC,Village_HHD_Cluster_BF,District_HHD_Cluster_BF,Village_HHD_Cluster_EMP,District_HHD_Cluster_EMP
Village_ID,,,,,,,,,,,,,,,,,,,,,
57138,57881,57881,6,69,357,57138,0,0,VILLAGE,Qasampur (251),...,2. Moderately-Developed,3. Developed,1. Under-Developed,1. Under-Developed,3. Developed,3. Developed,3. Developed,3. Developed,2. High AL,3. High Non-AL
57201,58017,58017,6,70,358,57201,0,0,VILLAGE,Babak Majra (51),...,2. Moderately-Developed,3. Developed,1. Under-Developed,1. Under-Developed,3. Developed,3. Developed,2. Moderately-Developed,3. Developed,1. High Unemployement,3. High Non-AL
57364,58175,58175,6,70,359,57364,0,0,VILLAGE,Sarai Mehadud (112),...,2. Moderately-Developed,3. Developed,3. Developed-,1. Under-Developed,1. Under-Developed,3. Developed,3. Developed,3. Developed,3. High Non-AL,3. High Non-AL
57375,58185,58185,6,70,359,57375,0,0,VILLAGE,Meranpur (106),...,2. Moderately-Developed,3. Developed,3. Developed-,1. Under-Developed,1. Under-Developed,3. Developed,1. Under-Developed,3. Developed,1. High Unemployement,3. High Non-AL
57944,58841,58841,6,71,361,57944,0,0,VILLAGE,Naugawan Sarkar (108),...,3. Developed,3. Developed,3. Developed-,2. Moderately-Developed,1. Under-Developed,3. Developed,3. Developed,3. Developed,3. High Non-AL,3. High Non-AL


In [ ]:
vill_csv.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'State', 'District', 'Subdistt',
       'Town/Village', 'Ward', 'EB', 'Level', 'Name', 'TRU', 'ELG_POP',
       'No_HH', 'BF_RUD', 'BF_INT', 'BF_ADV', 'CHH_RUD', 'CHH_INT', 'CHH_ADV',
       'FC_RUD', 'FC_INT', 'FC_ADV', 'MSL_INT', 'MSL_ADV', 'MSW_RUD',
       'MSW_INT', 'MSW_ADV', 'EMP_AL', 'EMP_NAL', 'EMP_UN', 'Village',
       'Village_HHD_Cluster_MSL', 'District_HHD_Cluster_MSL',
       'Village_HHD_Cluster_MSW', 'District_HHD_Cluster_MSW',
       'Village_HHD_Cluster_CHH', 'District_HHD_Cluster_CHH',
       'Village_HHD_Cluster_FC', 'District_HHD_Cluster_FC',
       'Village_HHD_Cluster_BF', 'District_HHD_Cluster_BF',
       'Village_HHD_Cluster_EMP', 'District_HHD_Cluster_EMP'],
      dtype='object')

In [ ]:
vill_df11 = pickle.load(open('/content/drive/MyDrive/Scripts_and_Data_Transfer/4.Training_CNN_(Arch_3)/Data/2011_PA_df.pickle', 'rb'))
print(len(vill_df11))
# vill_df11.head()

vill_df11['Village_ID'] = vill_df11['Town/Village']

vill_df11.set_index('Village_ID', inplace = True)
vill_df11 = vill_df11[vill_df11['Level'] != 'WARD']
vill_df11 = vill_df11[np.logical_not(vill_df11.index.duplicated())]

print(len(vill_df11))
vill_df11.head()

731602
648865


,State,District,Subdistt,Town/Village,Ward,EB,Level,Name,TRU,No_HH,...,MARG_AL_0_3_F,MARG_HH_0_3_P,MARG_HH_0_3_M,MARG_HH_0_3_F,MARG_OT_0_3_P,MARG_OT_0_3_M,MARG_OT_0_3_F,NON_WORK_P,NON_WORK_M,NON_WORK_F
Village_ID,,,,,,,,,,,,,,,,,,,,,
1,1,1,1,1,0,0,VILLAGE,Bore,Rural,130,...,0,0,0,0,0,0,0,0,0,0
2,1,1,1,2,0,0,VILLAGE,Keran,Rural,400,...,0,0,0,0,0,0,0,2616,1447,1169
3,1,1,1,3,0,0,VILLAGE,Bugna,Rural,0,...,0,0,0,0,0,0,0,0,0,0
4,1,1,1,4,0,0,VILLAGE,Bichwal,Rural,0,...,0,0,0,0,0,0,0,0,0,0
5,1,1,1,5,0,0,VILLAGE,Mindiyan,Rural,528,...,0,0,0,0,0,0,0,4561,2094,2467


In [ ]:
print(list(vill_df11.columns))

['State', 'District', 'Subdistt', 'Town/Village', 'Ward', 'EB', 'Level', 'Name', 'TRU', 'No_HH', 'TOT_P', 'TOT_M', 'TOT_F', 'P_06', 'M_06', 'F_06', 'P_SC', 'M_SC', 'F_SC', 'P_ST', 'M_ST', 'F_ST', 'P_LIT', 'M_LIT', 'F_LIT', 'P_ILL', 'M_ILL', 'F_ILL', 'TOT_WORK_P', 'TOT_WORK_M', 'TOT_WORK_F', 'MAINWORK_P', 'MAINWORK_M', 'MAINWORK_F', 'MAIN_CL_P', 'MAIN_CL_M', 'MAIN_CL_F', 'MAIN_AL_P', 'MAIN_AL_M', 'MAIN_AL_F', 'MAIN_HH_P', 'MAIN_HH_M', 'MAIN_HH_F', 'MAIN_OT_P', 'MAIN_OT_M', 'MAIN_OT_F', 'MARGWORK_P', 'MARGWORK_M', 'MARGWORK_F', 'MARG_CL_P', 'MARG_CL_M', 'MARG_CL_F', 'MARG_AL_P', 'MARG_AL_M', 'MARG_AL_F', 'MARG_HH_P', 'MARG_HH_M', 'MARG_HH_F', 'MARG_OT_P', 'MARG_OT_M', 'MARG_OT_F', 'MARGWORK_3_6_P', 'MARGWORK_3_6_M', 'MARGWORK_3_6_F', 'MARG_CL_3_6_P', 'MARG_CL_3_6_M', 'MARG_CL_3_6_F', 'MARG_AL_3_6_P', 'MARG_AL_3_6_M', 'MARG_AL_3_6_F', 'MARG_HH_3_6_P', 'MARG_HH_3_6_M', 'MARG_HH_3_6_F', 'MARG_OT_3_6_P', 'MARG_OT_3_6_M', 'MARG_OT_3_6_F', 'MARGWORK_0_3_P', 'MARGWORK_0_3_M', 'MARGWORK_0_3_F', 

In [ ]:
!pip install pyreadstat
import pyreadstat

vill_df01, meta = pyreadstat.read_dta('/content/drive/MyDrive/Satellite-Project-Village-level/2001IndiaCensus/India_villdir.dta')
vill_df01

,c_code01,ST_CODE,DIST_CODE,THSIL_CODE,BLOCK_CODE,V_CT_CODE,VILL_NAME,AREA,T_HH,T_P,T_M,T_F,SC_P,SC_M,SC_F,ST_P,ST_M,ST_F,EDU_FAC,P_SCH,RANG_P_SCH,M_SCH,RANG_M_SCH,S_SCH,S_S_SCH,COLLEGE,RANG_COLL,IND_SCH,TR_SCH,ADLT_LT_CT,OTH_SCH,MEDI_FAC,ALL_HOSP,RANG_ALL,AYU_HOSP,UN_HOSP,HOM_HOSP,ALL_DISP,AYU_DISP,UN_DISP,...,POWER_SUPL,POWER_DOM,POWER_AGR,POWER_OTH,POWER_ALL,PAP_MAG,NEWS_PAP,MAGAZINE,A_INCEXP,TOT_INC,TOT_EXP,MAN_COMM1,MAN_COMM2,MAN_COMM3,LAND_FORES,CANAL_GOVT,CANAL_PVT,WELL_WO_EL,WELL_W_EL,TW_WO_EL,TW_W_EL,TANK_IRR,RIVER_IRR,LAKE_IRR,W_FALL,OTH_IRR,TOT_IRR,UN_IRR,CULT_WASTE,AREA_NA_CU,DR,DR_WAT,BR,EL,EM,EN,JHUM_CUL,ROTATION_P,MAIN_CROP,state_ut
0,0101000100000200,01,01,0001,0001,00000200,Keran,237.55,178,2550,1670,880,1,1,0,1,1,0,1,5,0,2,0,1,0,0,3,0,0,0,0,1,0,3,0,0,0,0,0,0,...,2,0,0,0,0,2,,,2,0,0,,,,0,0,57.06,0,0,0,0,0,0,0,0,0,57.06,26.31,42.49,111.69,,,,,,,,,,Jammu & Kashmir
1,0101000100000500,01,01,0001,0001,00000500,Mindiyan,355.32,334,3913,1934,1979,0,0,0,0,0,0,1,4,0,1,0,0,0,0,3,0,0,0,0,1,0,3,0,0,0,0,0,0,...,2,0,0,0,0,2,,,2,0,0,,,,0,0,71.63,0,0,0,0,0,0,0,0,0,71.63,41.6,63.95,178.06,,,,,,,,,,Jammu & Kashmir
2,0101000100000600,01,01,0001,0001,00000600,Patrin,146.5,153,2423,1816,607,0,0,0,188,92,96,1,2,0,2,0,0,0,0,3,0,0,0,0,2,0,3,0,0,0,0,0,0,...,2,0,0,0,0,2,,,2,0,0,,,,0,0,20.23,0,0,0,0,0,0,0,0,0,20.23,38.45,38.8,48.97,,,,,,,,,,Jammu & Kashmir
3,0101000100000700,01,01,0001,0002,00000700,Juma Gund,130.3,101,740,385,355,0,0,0,253,138,115,1,2,0,1,0,0,0,0,3,0,0,0,0,1,0,3,0,0,0,0,0,1,...,2,0,0,0,0,2,,,2,0,0,,,,0,0,28.73,0,0,0,0,0,0,0,0,0,28.73,32.38,44.11,25.09,,,,,,,,,,Jammu & Kashmir
4,0101000100000800,01,01,0001,0002,00000800,Khantha Wali(Shalun Bhatta ),85,5,31,18,13,0,0,0,10,5,5,1,2,0,1,0,0,0,0,3,0,0,0,0,1,0,3,0,0,0,0,0,0,...,2,0,0,0,0,2,,,2,0,0,,,,0,0,27.11,0,0,0,0,0,0,0,0,0,27.11,7.28,21.86,28.73,,,,,,,,,,Jammu & Kashmir
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
584965,3502000200054000,35,02,0002,0002,00054000,Campbell Bay,752.36,1147,4113,2419,1694,0,0,0,195,119,76,1,3,0,2,0,2,2,0,3,0,0,0,0,1,0,3,0,0,0,1,0,0,...,1,1,2,2,2,1,N,M,2,0,0,Copra,-,-,,0,0,.02,,,,,,,,,,,,,,,,,,,,,,Andaman
584966,3502000200054100,35,02,0002,0002,00054100,Ranganathan Bay,,1,4,2,2,0,0,0,4,2,2,2,0,2,0,2,0,0,0,3,0,0,0,0,2,0,3,0,0,0,0,0,0,...,2,2,2,2,2,2,-,-,2,0,0,-,-,-,0,0,0,,,,,,,,,,,,,,,,,,,,,,,Andaman
584967,3502000200054400,35,02,0002,0002,00054400,Chaw Nallaha,,1,4,4,0,0,0,0,4,4,0,2,0,1,0,1,0,0,0,3,0,0,0,0,2,0,3,0,0,0,0,0,0,...,2,2,2,2,2,2,-,-,2,0,0,-,-,-,,0,0,,,,,,,,,,,,,,,,,,,,,,,Andaman
584968,3502000200054500,35,02,0002,0002,00054500,Navy Dera,,1,5,5,0,0,0,0,5,5,0,2,0,1,0,1,0,0,0,3,0,0,0,0,2,0,3,0,0,0,0,0,0,...,2,2,2,2,2,2,-,-,2,0,0,-,-,-,,0,0,,,,,,,,,,,,,,,,,,,,,,,Andaman


In [ ]:
vill_df01['Village_ID'] = pd.to_numeric(vill_df01['c_code01'], errors='coerce').astype('Int64')
vill_df01 = vill_df01.dropna()
vill_df01

,c_code01,ST_CODE,DIST_CODE,THSIL_CODE,BLOCK_CODE,V_CT_CODE,VILL_NAME,AREA,T_HH,T_P,T_M,T_F,SC_P,SC_M,SC_F,ST_P,ST_M,ST_F,EDU_FAC,P_SCH,RANG_P_SCH,M_SCH,RANG_M_SCH,S_SCH,S_S_SCH,COLLEGE,RANG_COLL,IND_SCH,TR_SCH,ADLT_LT_CT,OTH_SCH,MEDI_FAC,ALL_HOSP,RANG_ALL,AYU_HOSP,UN_HOSP,HOM_HOSP,ALL_DISP,AYU_DISP,UN_DISP,...,POWER_DOM,POWER_AGR,POWER_OTH,POWER_ALL,PAP_MAG,NEWS_PAP,MAGAZINE,A_INCEXP,TOT_INC,TOT_EXP,MAN_COMM1,MAN_COMM2,MAN_COMM3,LAND_FORES,CANAL_GOVT,CANAL_PVT,WELL_WO_EL,WELL_W_EL,TW_WO_EL,TW_W_EL,TANK_IRR,RIVER_IRR,LAKE_IRR,W_FALL,OTH_IRR,TOT_IRR,UN_IRR,CULT_WASTE,AREA_NA_CU,DR,DR_WAT,BR,EL,EM,EN,JHUM_CUL,ROTATION_P,MAIN_CROP,state_ut,Village_ID
0,0101000100000200,01,01,0001,0001,00000200,Keran,237.55,178,2550,1670,880,1,1,0,1,1,0,1,5,0,2,0,1,0,0,3,0,0,0,0,1,0,3,0,0,0,0,0,0,...,0,0,0,0,2,,,2,0,0,,,,0,0,57.06,0,0,0,0,0,0,0,0,0,57.06,26.31,42.49,111.69,,,,,,,,,,Jammu & Kashmir,101000100000200
1,0101000100000500,01,01,0001,0001,00000500,Mindiyan,355.32,334,3913,1934,1979,0,0,0,0,0,0,1,4,0,1,0,0,0,0,3,0,0,0,0,1,0,3,0,0,0,0,0,0,...,0,0,0,0,2,,,2,0,0,,,,0,0,71.63,0,0,0,0,0,0,0,0,0,71.63,41.6,63.95,178.06,,,,,,,,,,Jammu & Kashmir,101000100000500
2,0101000100000600,01,01,0001,0001,00000600,Patrin,146.5,153,2423,1816,607,0,0,0,188,92,96,1,2,0,2,0,0,0,0,3,0,0,0,0,2,0,3,0,0,0,0,0,0,...,0,0,0,0,2,,,2,0,0,,,,0,0,20.23,0,0,0,0,0,0,0,0,0,20.23,38.45,38.8,48.97,,,,,,,,,,Jammu & Kashmir,101000100000600
3,0101000100000700,01,01,0001,0002,00000700,Juma Gund,130.3,101,740,385,355,0,0,0,253,138,115,1,2,0,1,0,0,0,0,3,0,0,0,0,1,0,3,0,0,0,0,0,1,...,0,0,0,0,2,,,2,0,0,,,,0,0,28.73,0,0,0,0,0,0,0,0,0,28.73,32.38,44.11,25.09,,,,,,,,,,Jammu & Kashmir,101000100000700
4,0101000100000800,01,01,0001,0002,00000800,Khantha Wali(Shalun Bhatta ),85,5,31,18,13,0,0,0,10,5,5,1,2,0,1,0,0,0,0,3,0,0,0,0,1,0,3,0,0,0,0,0,0,...,0,0,0,0,2,,,2,0,0,,,,0,0,27.11,0,0,0,0,0,0,0,0,0,27.11,7.28,21.86,28.73,,,,,,,,,,Jammu & Kashmir,101000100000800
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
584965,3502000200054000,35,02,0002,0002,00054000,Campbell Bay,752.36,1147,4113,2419,1694,0,0,0,195,119,76,1,3,0,2,0,2,2,0,3,0,0,0,0,1,0,3,0,0,0,1,0,0,...,1,2,2,2,1,N,M,2,0,0,Copra,-,-,,0,0,.02,,,,,,,,,,,,,,,,,,,,,,Andaman,3502000200054000
584966,3502000200054100,35,02,0002,0002,00054100,Ranganathan Bay,,1,4,2,2,0,0,0,4,2,2,2,0,2,0,2,0,0,0,3,0,0,0,0,2,0,3,0,0,0,0,0,0,...,2,2,2,2,2,-,-,2,0,0,-,-,-,0,0,0,,,,,,,,,,,,,,,,,,,,,,,Andaman,3502000200054100
584967,3502000200054400,35,02,0002,0002,00054400,Chaw Nallaha,,1,4,4,0,0,0,0,4,4,0,2,0,1,0,1,0,0,0,3,0,0,0,0,2,0,3,0,0,0,0,0,0,...,2,2,2,2,2,-,-,2,0,0,-,-,-,,0,0,,,,,,,,,,,,,,,,,,,,,,,Andaman,3502000200054400
584968,3502000200054500,35,02,0002,0002,00054500,Navy Dera,,1,5,5,0,0,0,0,5,5,0,2,0,1,0,1,0,0,0,3,0,0,0,0,2,0,3,0,0,0,0,0,0,...,2,2,2,2,2,-,-,2,0,0,-,-,-,,0,0,,,,,,,,,,,,,,,,,,,,,,,Andaman,3502000200054500


In [ ]:
vill_df01.set_index('Village_ID', inplace = True)
vill_df01 = vill_df01[np.logical_not(vill_df01.index.duplicated())]

IN01 = pickle.load(open('/content/drive/MyDrive/Scripts_and_Data_Transfer/3.Arch_2_Procedure/5.Train_Regression_and_Classification_Models/Inputs/Common_Indicators_Inputs_(With_addtional_village_size_and_population)/Input_2001.pickle','rb'))

print(len(vill_df01))
vill_df01.head()

584969


,c_code01,ST_CODE,DIST_CODE,THSIL_CODE,BLOCK_CODE,V_CT_CODE,VILL_NAME,AREA,T_HH,T_P,T_M,T_F,SC_P,SC_M,SC_F,ST_P,ST_M,ST_F,EDU_FAC,P_SCH,RANG_P_SCH,M_SCH,RANG_M_SCH,S_SCH,S_S_SCH,COLLEGE,RANG_COLL,IND_SCH,TR_SCH,ADLT_LT_CT,OTH_SCH,MEDI_FAC,ALL_HOSP,RANG_ALL,AYU_HOSP,UN_HOSP,HOM_HOSP,ALL_DISP,AYU_DISP,UN_DISP,...,POWER_SUPL,POWER_DOM,POWER_AGR,POWER_OTH,POWER_ALL,PAP_MAG,NEWS_PAP,MAGAZINE,A_INCEXP,TOT_INC,TOT_EXP,MAN_COMM1,MAN_COMM2,MAN_COMM3,LAND_FORES,CANAL_GOVT,CANAL_PVT,WELL_WO_EL,WELL_W_EL,TW_WO_EL,TW_W_EL,TANK_IRR,RIVER_IRR,LAKE_IRR,W_FALL,OTH_IRR,TOT_IRR,UN_IRR,CULT_WASTE,AREA_NA_CU,DR,DR_WAT,BR,EL,EM,EN,JHUM_CUL,ROTATION_P,MAIN_CROP,state_ut
Village_ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
101000100000200,0101000100000200,01,01,0001,0001,00000200,Keran,237.55,178,2550,1670,880,1,1,0,1,1,0,1,5,0,2,0,1,0,0,3,0,0,0,0,1,0,3,0,0,0,0,0,0,...,2,0,0,0,0,2,,,2,0,0,,,,0,0,57.06,0,0,0,0,0,0,0,0,0,57.06,26.31,42.49,111.69,,,,,,,,,,Jammu & Kashmir
101000100000500,0101000100000500,01,01,0001,0001,00000500,Mindiyan,355.32,334,3913,1934,1979,0,0,0,0,0,0,1,4,0,1,0,0,0,0,3,0,0,0,0,1,0,3,0,0,0,0,0,0,...,2,0,0,0,0,2,,,2,0,0,,,,0,0,71.63,0,0,0,0,0,0,0,0,0,71.63,41.6,63.95,178.06,,,,,,,,,,Jammu & Kashmir
101000100000600,0101000100000600,01,01,0001,0001,00000600,Patrin,146.5,153,2423,1816,607,0,0,0,188,92,96,1,2,0,2,0,0,0,0,3,0,0,0,0,2,0,3,0,0,0,0,0,0,...,2,0,0,0,0,2,,,2,0,0,,,,0,0,20.23,0,0,0,0,0,0,0,0,0,20.23,38.45,38.8,48.97,,,,,,,,,,Jammu & Kashmir
101000100000700,0101000100000700,01,01,0001,0002,00000700,Juma Gund,130.3,101,740,385,355,0,0,0,253,138,115,1,2,0,1,0,0,0,0,3,0,0,0,0,1,0,3,0,0,0,0,0,1,...,2,0,0,0,0,2,,,2,0,0,,,,0,0,28.73,0,0,0,0,0,0,0,0,0,28.73,32.38,44.11,25.09,,,,,,,,,,Jammu & Kashmir
101000100000800,0101000100000800,01,01,0001,0002,00000800,Khantha Wali(Shalun Bhatta ),85,5,31,18,13,0,0,0,10,5,5,1,2,0,1,0,0,0,0,3,0,0,0,0,1,0,3,0,0,0,0,0,0,...,2,0,0,0,0,2,,,2,0,0,,,,0,0,27.11,0,0,0,0,0,0,0,0,0,27.11,7.28,21.86,28.73,,,,,,,,,,Jammu & Kashmir


In [ ]:
vill_df01[vill_df01.index==2001000100000600]

,c_code01,ST_CODE,DIST_CODE,THSIL_CODE,BLOCK_CODE,V_CT_CODE,VILL_NAME,AREA,T_HH,T_P,T_M,T_F,SC_P,SC_M,SC_F,ST_P,ST_M,ST_F,EDU_FAC,P_SCH,RANG_P_SCH,M_SCH,RANG_M_SCH,S_SCH,S_S_SCH,COLLEGE,RANG_COLL,IND_SCH,TR_SCH,ADLT_LT_CT,OTH_SCH,MEDI_FAC,ALL_HOSP,RANG_ALL,AYU_HOSP,UN_HOSP,HOM_HOSP,ALL_DISP,AYU_DISP,UN_DISP,...,POWER_SUPL,POWER_DOM,POWER_AGR,POWER_OTH,POWER_ALL,PAP_MAG,NEWS_PAP,MAGAZINE,A_INCEXP,TOT_INC,TOT_EXP,MAN_COMM1,MAN_COMM2,MAN_COMM3,LAND_FORES,CANAL_GOVT,CANAL_PVT,WELL_WO_EL,WELL_W_EL,TW_WO_EL,TW_W_EL,TANK_IRR,RIVER_IRR,LAKE_IRR,W_FALL,OTH_IRR,TOT_IRR,UN_IRR,CULT_WASTE,AREA_NA_CU,DR,DR_WAT,BR,EL,EM,EN,JHUM_CUL,ROTATION_P,MAIN_CROP,state_ut
Village_ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2001000100000600,2001000100000600,20,01,0001,0001,00000600,Sundi,564,362,2221,1179,1042,470,239,231,542,280,262,1,1,0,0,2,0,0,0,3,0,0,0,0,2,0,3,0,0,0,0,0,0,...,2,0,0,0,0,2,,,2,0,0,GUR,BASKET,,260.63,0,0,24,0,0,0,0,126,0,0,0,150,86,35.2,32.45,,,,,,,,,,Jharkhand


In [ ]:
mappingx = pickle.load(open('/content/drive/MyDrive/Scripts_and_Data_Transfer/4.Training_CNN_(Arch_3)/Data/village_code_mapping/2001_2011_corr.pickle', 'rb'))
mappingy = pickle.load(open('/content/drive/MyDrive/Scripts_and_Data_Transfer/4.Training_CNN_(Arch_3)/Data/village_code_mapping/2011_2001_corr.pickle', 'rb'))
print(type(list(mappingx.keys())[0]), type(list(mappingy.keys())[0]))
print(type(mappingx[2011000501968600]), type(mappingy[370730]))

<class 'int'> <class 'int'>
<class 'int'> <class 'str'>


In [ ]:
IN01

,BF_0_OUT_1,BF_0_OUT_2,BF_0_OUT_3,FC_0_OUT_1,FC_0_OUT_2,FC_0_OUT_3,MSW_0_OUT_1,MSW_0_OUT_2,MSW_0_OUT_3,ASSET_0_OUT_1,ASSET_0_OUT_2,ASSET_0_OUT_3,BF_1_OUT_1,BF_1_OUT_2,BF_1_OUT_3,FC_1_OUT_1,FC_1_OUT_2,FC_1_OUT_3,MSW_1_OUT_1,MSW_1_OUT_2,MSW_1_OUT_3,ASSET_1_OUT_1,ASSET_1_OUT_2,ASSET_1_OUT_3,BF_2_OUT_1,BF_2_OUT_2,BF_2_OUT_3,FC_2_OUT_1,FC_2_OUT_2,FC_2_OUT_3,MSW_2_OUT_1,MSW_2_OUT_2,MSW_2_OUT_3,ASSET_2_OUT_1,ASSET_2_OUT_2,ASSET_2_OUT_3,BF_3_OUT_1,BF_3_OUT_2,BF_3_OUT_3,FC_3_OUT_1,...,dc_score_none_size_intensity,dc_score_log_none,dc_score_log_size,dc_score_log_size_intensity,dc_score_sqrt_none,dc_score_sqrt_size,dc_score_sqrt_size_intensity,dc_score_none_intensity,dc_score_log_intensity,dc_score_sqrt_intensity,BF_N_OUT_1,BF_N_OUT_2,BF_N_OUT_3,FC_N_OUT_1,FC_N_OUT_2,FC_N_OUT_3,MSW_N_OUT_1,MSW_N_OUT_2,MSW_N_OUT_3,ASSET_N_OUT_1,ASSET_N_OUT_2,ASSET_N_OUT_3,dc_score_none_size_only,dc_score_none_size_intensity_only,dc_score_none_intensity_only,dc_score_log_size_only,dc_score_log_size_intensity_only,dc_score_log_intensity_only,dc_score_sqrt_size_only,dc_score_sqrt_size_intensity_only,dc_score_sqrt_intensity_only,pop,hh,log_pop,sqrt_pop,log_hh,sqrt_hh,area,log_area,sqrt_area
Village_ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2001000100000200,0.713842,0.285233,0.000925,0.966589,0.032734,0.000678,0.333333,0.333333,0.333333,0.671659,0.327996,0.000345,0.022118,0.976707,0.001175,0.333333,0.333333,0.333333,0.333333,0.333333,0.333333,0.480562,0.508362,0.011075,0.281899,0.717452,0.000649,0.907609,0.088726,0.003664,0.333333,0.333333,0.333333,0.084197,0.844209,0.071593,0.043549,0.948645,0.007806,0.333333,...,-0.531541,-1.124740,-0.647166,-0.797238,-0.873864,-0.603929,-0.699803,-0.258189,-1.050285,-0.778489,0.270843,0.726724,0.002433,0.548393,0.246490,0.205117,0.333333,0.333333,0.333333,0.166329,0.774889,0.058782,-1.078912,0.885764,2.745341,-0.823889,0.721405,1.375449,-1.026595,0.827574,2.086867,-0.507038,-0.572465,-0.352010,-0.543928,-0.600717,-0.713863,0.052758,0.474085,0.269818
2001000100000300,0.022118,0.976707,0.001175,0.333333,0.333333,0.333333,0.333333,0.333333,0.333333,0.480562,0.508362,0.011075,0.713842,0.285233,0.000925,0.966589,0.032734,0.000678,0.333333,0.333333,0.333333,0.671659,0.327996,0.000345,0.281899,0.717452,0.000649,0.907609,0.088726,0.003664,0.333333,0.333333,0.333333,0.084197,0.844209,0.071593,0.574995,0.421756,0.003249,0.969722,...,-0.532500,-1.192420,-0.686133,-0.800699,-0.906934,-0.628640,-0.701855,-0.257887,-1.047448,-0.777015,0.490165,0.507123,0.002713,0.702117,0.163560,0.134323,0.460879,0.269744,0.269377,0.238182,0.714045,0.047773,-1.078912,0.885764,2.745341,-0.823889,0.721405,1.375449,-1.026595,0.827574,2.086867,-0.395274,-0.437516,-0.108836,-0.340541,-0.229352,-0.436619,0.390641,0.778430,0.624496
2001000100000400,0.281899,0.717452,0.000649,0.907609,0.088726,0.003664,0.333333,0.333333,0.333333,0.084197,0.844209,0.071593,0.043549,0.948645,0.007806,0.333333,0.333333,0.333333,0.333333,0.333333,0.333333,0.008791,0.977917,0.013292,0.713842,0.285233,0.000925,0.966589,0.032734,0.000678,0.333333,0.333333,0.333333,0.671659,0.327996,0.000345,0.022118,0.976707,0.001175,0.333333,...,-0.532695,-1.102438,-0.643393,-0.801405,-0.862770,-0.601520,-0.702273,-0.258161,-1.050024,-0.778353,0.357232,0.640280,0.002488,0.560189,0.235291,0.204520,0.333333,0.333333,0.333333,0.283822,0.671646,0.044532,-1.078912,0.885764,2.745341,-0.823889,0.721405,1.375449,-1.026595,0.827574,2.086867,0.457625,0.267223,0.835745,0.756012,0.709298,0.562130,1.425677,1.399927,1.509602
2001000100000500,0.043549,0.948645,0.007806,0.333333,0.333333,0.333333,0.333333,0.333333,0.333333,0.008791,0.977917,0.013292,0.281899,0.717452,0.000649,0.907609,0.088726,0.003664,0.333333,0.333333,0.333333,0.084197,0.844209,0.071593,0.170113,0.828287,0.001600,0.834357,0.143722,0.021921,0.333333,0.333333,0.333333,0.147248,0.806394,0.046357,0.836538,0.162528,0.000934,0.333333,...,-0.532696,-1.038383,-0.612023,-0.801406,-0.830356,-0.581374,-0.702273,-0.258

In [ ]:
IN11

,BF_0_OUT_1,BF_0_OUT_2,BF_0_OUT_3,FC_0_OUT_1,FC_0_OUT_2,FC_0_OUT_3,MSW_0_OUT_1,MSW_0_OUT_2,MSW_0_OUT_3,BF_1_OUT_1,BF_1_OUT_2,BF_1_OUT_3,FC_1_OUT_1,FC_1_OUT_2,FC_1_OUT_3,MSW_1_OUT_1,MSW_1_OUT_2,MSW_1_OUT_3,BF_2_OUT_1,BF_2_OUT_2,BF_2_OUT_3,FC_2_OUT_1,FC_2_OUT_2,FC_2_OUT_3,MSW_2_OUT_1,MSW_2_OUT_2,MSW_2_OUT_3,BF_3_OUT_1,BF_3_OUT_2,BF_3_OUT_3,FC_3_OUT_1,FC_3_OUT_2,FC_3_OUT_3,MSW_3_OUT_1,MSW_3_OUT_2,MSW_3_OUT_3,BF_4_OUT_1,BF_4_OUT_2,BF_4_OUT_3,FC_4_OUT_1,...,ASSET_1_OUT_4,ASSET_2_OUT_4,ASSET_3_OUT_4,ASSET_4_OUT_4,ASSET_5_OUT_4,ASSET_N_OUT_4,LIT_0_OUT_1,LIT_1_OUT_1,LIT_2_OUT_1,LIT_3_OUT_1,LIT_4_OUT_1,LIT_5_OUT_1,LIT_N_OUT_1,dc_score_none_none,dc_score_none_size,dc_score_none_size_intensity,dc_score_none_intensity,dc_score_log_none,dc_score_log_size,dc_score_log_size_intensity,dc_score_log_intensity,dc_score_sqrt_none,dc_score_sqrt_size,dc_score_sqrt_size_intensity,dc_score_sqrt_intensity,dc_score_none_size_only,dc_score_none_size_intensity_only,dc_score_none_intensity_only,dc_score_log_size_only,dc_score_log_size_intensity_only,dc_score_log_intensity_only,dc_score_sqrt_size_only,dc_score_sqrt_size_intensity_only,dc_score_sqrt_intensity_only,pop,hh,log_pop,sqrt_pop,log_hh,sqrt_hh
301000100000500,0.450654,0.252962,0.296384,0.503688,0.228672,0.267641,0.283980,0.416766,0.299254,0.404751,0.341285,0.253963,0.516307,0.223787,0.259906,0.302083,0.329116,0.368801,0.433088,0.307435,0.259478,0.485192,0.219216,0.295592,0.276847,0.337818,0.385335,0.411186,0.296979,0.291835,0.514276,0.228837,0.256888,0.321716,0.249432,0.428852,0.353726,0.320924,0.325350,0.462787,...,0.869074,0.891126,0.797312,0.526659,1.131836,0.933604,1.123079,1.375410,1.281753,1.010488,1.402736,0.961566,1.490337,-0.010794,-0.235655,-0.118069,0.997633,0.405644,-0.065347,0.220358,1.767178,0.178014,-0.201343,0.025999,1.726904,0.971891,0.586182,2.305343,0.751182,0.483776,1.696890,0.889434,0.550484,2.077334,-0.347677,-0.323749,-0.010608,-0.258938,0.007857,-0.234668
301000100000600,0.455719,0.267446,0.276835,0.521914,0.222991,0.255094,0.273379,0.385237,0.341383,0.411186,0.296979,0.291835,0.514276,0.228837,0.256888,0.321716,0.249432,0.428852,0.373860,0.334283,0.291856,0.478079,0.225658,0.296263,0.285217,0.270897,0.443887,0.429027,0.269969,0.301004,0.478039,0.229054,0.292907,0.310845,0.287189,0.401965,0.496368,0.242131,0.261501,0.366809,...,0.802736,0.669899,1.128989,0.833902,0.823613,0.943285,1.723978,1.012821,1.591440,0.964275,0.578266,0.493267,1.144557,-0.073575,-0.235961,-0.118457,1.108978,0.199958,-0.066059,0.219650,1.861324,0.007298,-0.201895,0.025414,1.863071,0.971891,0.586182,2.305343,0.751182,0.483776,1.696890,0.889434,0.550484,2.077334,-0.384031,-0.359134,-0.075442,-0.319619,-0.056165,-0.294303
301000100000700,0.433088,0.307435,0.259478,0.485192,0.219216,0.295592,0.276847,0.337818,0.385335,0.404751,0.341285,0.253963,0.516307,0.223787,0.259906,0.302083,0.329116,0.368801,0.450654,0.252962,0.296384,0.503688,0.228672,0.267641,0.283980,0.416766,0.299254,0.411186,0.296979,0.291835,0.514276,0.228837,0.256888,0.321716,0.249432,0.428852,0.353726,0.320924,0.325350,0.462787,...,0.869074,1.164270,0.797312,0.526659,1.131836,0.994104,1.422714,1.375410,1.019074,1.010488,1.402736,0.961566,1.425692,-0.047625,-0.231038,-0.112210,0.966522,0.288532,-0.054641,0.231014,1.739732,0.079335,-0.193022,0.034803,1.688040,0.971891,0.586182,2.305343,0.751182,0.483776,1.696890,0.889434,0.550484,2.077334,0.677231,0.655232,0.965825,0.976497,0.975670,0.971300
301000100000800,0.404751,0.341285,0.253963,0.516307,0.223787,0.259906,0.302083,0.329116,0.368801,0.411186,0.296979,0.291835,0.514276,0.228837,0.256888,0.321716,0.249432,0.428852,0.450654,0.252962,0.296384,0.503688,0.228672,0.267641,0.283980,0.416766,0.299254,0.433088,0.307435,0.259478,0.485192,0.219216,0.295592,0.276847,0.337818,0.385335,0.353726,0.320924,0.325350,0.462787,...,0.802736,1.164270,0.889716,0.526659,1.131836,0.999923,1.526776,1.012821,1.019074,1.278734,1.402736,0.961566,1.403241,-0.091262,-0.233134,-0.114870,1.104069,0.136341,-0.059491,0.226187

In [ ]:
population, household = 'T_P', 'T_HH'
vill_df = vill_df01.copy()
IN = IN11.copy()

if year == '2011' :
  population, household = 'TOT_P', 'No_HH'
  vill_df = vill_df11.copy()
  IN = IN11.copy()

final_pop_2011 = vill_df[[population, household]]
final_pop_2011.rename(columns = {population:'pop', household : 'hh'}, inplace = True)

final_pop_2011 = final_pop_2011.filter(items = IN.index, axis=0)
final_pop_2011

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,pop,hh
Village_ID,,
301000100000500,766,119
301000100000600,730,116
301000100000700,2104,395
301000100000800,3014,472
301000100000900,461,70
...,...,...
3310000700779700,1365,403
3310000700781800,2054,581
3310000700781900,2388,687


In [ ]:
if year == '2001':
  mapping = pickle.load(open('/content/drive/MyDrive/Scripts_and_Data_Transfer/4.Training_CNN_(Arch_3)/Data/village_code_mapping/2001_2011_corr.pickle', 'rb'))
  final_pop_2011 = final_pop_2011.copy().rename(index=mapping)



In [ ]:
  final_pop_2011

,pop,hh
Village_ID,,
27378,766,119
27379,730,116
27380,2104,395
27381,3014,472
27382,461,70
...,...,...
644661,1365,403
644682,2054,581
644683,2388,687


In [ ]:
final_pop_2011 = final_pop_2011.astype(float)

In [ ]:
final_pop_2011['log_pop'] = np.log2(final_pop_2011['pop'])
final_pop_2011['sqrt_pop'] = np.sqrt(final_pop_2011['pop'])

final_pop_2011['log_hh'] = np.log2(final_pop_2011['hh'])
final_pop_2011['sqrt_hh'] = np.sqrt(final_pop_2011['hh'])

In [ ]:
final_pop_2011

,pop,hh,log_pop,sqrt_pop,log_hh,sqrt_hh
Village_ID,,,,,,
27378,766.0,119.0,9.581201,27.676705,6.894818,10.908712
27379,730.0,116.0,9.511753,27.018512,6.857981,10.770330
27380,2104.0,395.0,11.038919,45.869380,8.625709,19.874607
27381,3014.0,472.0,11.557464,54.899909,8.882643,21.725561
27382,461.0,70.0,8.848623,21.470911,6.129283,8.366600
...,...,...,...,...,...,...
644661,1365.0,403.0,10.414685,36.945906,8.654636,20.074860
644682,2054.0,581.0,11.004220,45.321077,9.182394,24.103942
644683,2388.0,687.0,11.221587,48.867167,9.424166,26.210685


In [ ]:
final_pop_2011.describe()

,pop,hh,log_pop,sqrt_pop,log_hh,sqrt_hh
count,315909.000000,315909.000000,315909.000000,315909.000000,315909.000000,315909.000000
mean,1267.866316,228.279343,9.618686,31.866857,7.146336,13.505063
std,1434.003906,266.946846,1.546981,15.886175,1.525273,6.774419
min,1.000000,1.000000,0.000000,1.000000,0.000000,1.000000
25%,445.000000,80.000000,8.797662,21.095023,6.321928,8.944272
50%,856.000000,153.000000,9.741467,29.257478,7.257388,12.369317
75%,1576.000000,280.000000,10.622052,39.698866,8.129283,16.733201
max,56450.000000,11391.000000,15.784686,237.592087,13.475607,106.728628


In [ ]:
pickle.dump(final_pop_2011, open('/content/drive/MyDrive/Scripts_and_Data_Transfer/3.Arch_2_Procedure/5.Train_Regression_and_Classification_Models/Inputs/population_data/population_'+year+'_raw.pickle', 'wb'))

In [ ]:
X= final_pop_2011.copy().to_numpy().astype(float)

In [ ]:
for i in range(0,6):
  print(i)
  X[:,i] = StandardScaler().fit_transform(X[:,i].reshape(-1,1)).reshape(-1)

0
1
2
3
4
5


In [ ]:
X = pd.DataFrame(X, index=final_pop_2011.index.copy(), columns = final_pop_2011.columns)

In [ ]:
X

,pop,hh,log_pop,sqrt_pop,log_hh,sqrt_hh
Village_ID,,,,,,
27378,-0.349976,-0.409368,-0.024231,-0.263761,-0.164901,-0.383259
27379,-0.375081,-0.420606,-0.069124,-0.305193,-0.189052,-0.403686
27380,0.583077,0.624547,0.918069,0.881430,0.969909,0.940236
27381,1.217665,0.912995,1.253267,1.449883,1.138360,1.213464
27382,-0.562668,-0.592926,-0.497785,-0.654403,-0.666802,-0.758511
...,...,...,...,...,...,...
644661,0.067736,0.654516,0.514551,0.319716,0.988874,0.969796
644682,0.548210,1.321316,0.895639,0.846915,1.334884,1.564547
644683,0.781124,1.718400,1.036150,1.070134,1.493395,1.875532


In [ ]:
X.describe()

,pop,hh,log_pop,sqrt_pop,log_hh,sqrt_hh
count,3.159090e+05,3.159090e+05,3.159090e+05,3.159090e+05,3.159090e+05,3.159090e+05
mean,-1.897074e-16,-1.078208e-15,-1.895654e-15,2.391432e-15,6.944365e-16,-1.954963e-15
std,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00
min,-8.834483e-01,-8.514043e-01,-6.217725e+00,-1.943004e+00,-4.685292e+00,-1.845927e+00
25%,-5.738252e-01,-5.554648e-01,-5.307279e-01,-6.780645e-01,-5.404997e-01,-6.732384e-01
50%,-2.872147e-01,-2.820017e-01,7.936819e-02,-1.642550e-01,7.280790e-02,-1.676525e-01
75%,2.148768e-01,1.937492e-01,6.485971e-01,4.930087e-01,6.444412e-01,4.765194e-01
max,3.848122e+01,4.181633e+01,3.985834e+00,1.294997e+01,4.149606e+00,1.376114e+01


In [ ]:
pickle.dump(X, open('/content/drive/MyDrive/Scripts_and_Data_Transfer/3.Arch_2_Procedure/5.Train_Regression_and_Classification_Models/Inputs/population_data/population_'+year+'.pickle', 'wb'))

#Creating 2019 population

In [ ]:
pop_2001 = pickle.load(open('/content/drive/MyDrive/Scripts_and_Data_Transfer/3.Arch_2_Procedure/5.Train_Regression_and_Classification_Models/Inputs/population_data/population_2001_raw.pickle', 'rb'))
# final_pop_2011.equals(pop_2001)

In [ ]:
pop_2011 = pickle.load(open('/content/drive/MyDrive/Scripts_and_Data_Transfer/3.Arch_2_Procedure/5.Train_Regression_and_Classification_Models/Inputs/population_data/population_2011_raw.pickle', 'rb'))
# final_pop_2011.equals(pop_2011)

In [ ]:
pop_2011 = pop_2011.filter(items = pop_2001.index, axis = 0)
pop_2001 = pop_2001.filter(items = pop_2011.index, axis = 0)

In [ ]:
pop_2011.rename(columns = {'pop' : 'total_pop_2011', 'hh' : 'total_hh_2011'}, inplace = True)
pop_2011

,total_pop_2011,total_hh_2011,log_pop,sqrt_pop,log_hh,sqrt_hh
Village_ID,,,,,,
27378,852.0,179.0,9.734710,29.189039,7.483816,13.379088
27379,780.0,167.0,9.607330,27.928480,7.383704,12.922848
27380,2502.0,511.0,11.288866,50.019996,8.997179,22.605309
27381,1887.0,376.0,10.881879,43.439613,8.554589,19.390719
27382,2233.0,382.0,11.124768,47.254629,8.577429,19.544820
...,...,...,...,...,...,...
644661,1291.0,438.0,10.334273,35.930488,8.774787,20.928450
644682,8089.0,2332.0,12.981746,89.938868,11.187352,48.290786
644683,3992.0,1315.0,11.962896,63.182276,10.360847,36.262929


In [ ]:
pop_2001.rename(columns = {'pop' : 'total_pop_2001', 'hh' : 'total_hh_2001'}, inplace = True)
pop_2001

,total_pop_2001,total_hh_2001,log_pop,sqrt_pop,log_hh,sqrt_hh
Village_ID,,,,,,
27378,766.0,119.0,9.581201,27.676705,6.894818,10.908712
27379,730.0,116.0,9.511753,27.018512,6.857981,10.770330
27380,2104.0,395.0,11.038919,45.869380,8.625709,19.874607
27381,3014.0,472.0,11.557464,54.899909,8.882643,21.725561
27382,461.0,70.0,8.848623,21.470911,6.129283,8.366600
...,...,...,...,...,...,...
644661,1365.0,403.0,10.414685,36.945906,8.654636,20.074860
644682,2054.0,581.0,11.004220,45.321077,9.182394,24.103942
644683,2388.0,687.0,11.221587,48.867167,9.424166,26.210685


In [ ]:
pop_final = pop_2011.copy()[['total_pop_2011','total_hh_2011']].join(pop_2001.copy()[['total_pop_2001','total_hh_2001']], how = 'inner')
pop_final

,total_pop_2011,total_hh_2011,total_pop_2001,total_hh_2001
Village_ID,,,,
27378,852.0,179.0,766.0,119.0
27379,780.0,167.0,730.0,116.0
27380,2502.0,511.0,2104.0,395.0
27381,1887.0,376.0,3014.0,472.0
27382,2233.0,382.0,461.0,70.0
...,...,...,...,...
644661,1291.0,438.0,1365.0,403.0
644682,8089.0,2332.0,2054.0,581.0
644683,3992.0,1315.0,2388.0,687.0


In [ ]:
combined_data_with_2019 = pop_final.copy()

In [ ]:
combined_data_with_2019['rate_of_population_growth_2001_to_2011'] = (combined_data_with_2019['total_pop_2011'] - combined_data_with_2019['total_pop_2001']) / combined_data_with_2019['total_pop_2001']
combined_data_with_2019['rate_of_household_growth_2001_to_2011'] = (combined_data_with_2019['total_hh_2011'] - combined_data_with_2019['total_hh_2001']) / combined_data_with_2019['total_hh_2001']

combined_data_with_2019

,total_pop_2011,total_hh_2011,total_pop_2001,total_hh_2001,rate_of_population_growth_2001_to_2011,rate_of_household_growth_2001_to_2011
Village_ID,,,,,,
27378,852.0,179.0,766.0,119.0,0.112272,0.504202
27379,780.0,167.0,730.0,116.0,0.068493,0.439655
27380,2502.0,511.0,2104.0,395.0,0.189163,0.293671
27381,1887.0,376.0,3014.0,472.0,-0.373922,-0.203390
27382,2233.0,382.0,461.0,70.0,3.843818,4.457143
...,...,...,...,...,...,...
644661,1291.0,438.0,1365.0,403.0,-0.054212,0.086849
644682,8089.0,2332.0,2054.0,581.0,2.938169,3.013769
644683,3992.0,1315.0,2388.0,687.0,0.671692,0.914119


In [ ]:
combined_data_with_2019['total_pop_2019'] = (1 + combined_data_with_2019['rate_of_population_growth_2001_to_2011']) * combined_data_with_2019['total_pop_2011']
combined_data_with_2019['total_hh_2019'] = (1 + combined_data_with_2019['rate_of_household_growth_2001_to_2011']) * combined_data_with_2019['total_hh_2011']
combined_data_with_2019

,total_pop_2011,total_hh_2011,total_pop_2001,total_hh_2001,rate_of_population_growth_2001_to_2011,rate_of_household_growth_2001_to_2011,total_pop_2019,total_hh_2019
Village_ID,,,,,,,,
27378,852.0,179.0,766.0,119.0,0.112272,0.504202,947.655352,269.252101
27379,780.0,167.0,730.0,116.0,0.068493,0.439655,833.424658,240.422414
27380,2502.0,511.0,2104.0,395.0,0.189163,0.293671,2975.287072,661.065823
27381,1887.0,376.0,3014.0,472.0,-0.373922,-0.203390,1181.409754,299.525424
27382,2233.0,382.0,461.0,70.0,3.843818,4.457143,10816.245119,2084.628571
...,...,...,...,...,...,...,...,...
644661,1291.0,438.0,1365.0,403.0,-0.054212,0.086849,1221.011722,476.039702
644682,8089.0,2332.0,2054.0,581.0,2.938169,3.013769,31855.852483,9360.110155
644683,3992.0,1315.0,2388.0,687.0,0.671692,0.914119,6673.393635,2517.066958


In [ ]:
pop_2019 = combined_data_with_2019.copy()[['total_pop_2019', 'total_hh_2019']]
pop_2019.describe()

,total_pop_2019,total_hh_2019
count,3.159090e+05,3.159090e+05
mean,2.130500e+03,4.324243e+02
std,1.103743e+05,8.701069e+03
min,3.597122e-03,3.623188e-03
25%,6.120209e+02,1.351406e+02
50%,1.194627e+03,2.595071e+02
75%,2.221939e+03,4.750045e+02
max,6.097267e+07,4.191372e+06


In [ ]:
pop_2019.rename(columns = {'total_pop_2019' : 'pop', 'total_hh_2019' : 'hh'}, inplace = True)
pop_2019

,pop,hh
Village_ID,,
27378,947.655352,269.252101
27379,833.424658,240.422414
27380,2975.287072,661.065823
27381,1181.409754,299.525424
27382,10816.245119,2084.628571
...,...,...
644661,1221.011722,476.039702
644682,31855.852483,9360.110155
644683,6673.393635,2517.066958


In [ ]:
final_pop_2019 = pop_2019.copy()

In [ ]:
final_pop_2019['log_pop'] = np.log2(final_pop_2019['pop'])
final_pop_2019['sqrt_pop'] = np.sqrt(final_pop_2019['pop'])

final_pop_2019['log_hh'] = np.log2(final_pop_2019['hh'])
final_pop_2019['sqrt_hh'] = np.sqrt(final_pop_2019['hh'])

In [ ]:
final_pop_2019

,pop,hh,log_pop,sqrt_pop,log_hh,sqrt_hh
Village_ID,,,,,,
27378,947.655352,269.252101,9.888219,30.784011,8.072814,16.408903
27379,833.424658,240.422414,9.702908,28.869095,7.909428,15.505561
27380,2975.287072,661.065823,11.538813,54.546192,9.368650,25.711200
27381,1181.409754,299.525424,10.206294,34.371642,8.226535,17.306803
27382,10816.245119,2084.628571,13.400912,104.001178,11.025575,45.657733
...,...,...,...,...,...,...
644661,1221.011722,476.039702,10.253861,34.942978,8.894938,21.818334
644682,31855.852483,9360.110155,14.959271,178.482079,13.192310,96.747662
644683,6673.393635,2517.066958,12.704205,81.690842,11.297528,50.170379


In [ ]:
final_pop_2019.describe()

,pop,hh,log_pop,sqrt_pop,log_hh,sqrt_hh
count,3.159090e+05,3.159090e+05,315909.000000,315909.000000,315909.000000,315909.000000
mean,2.130500e+03,4.324243e+02,10.095300,38.034894,7.899472,17.688122
std,1.103743e+05,8.701069e+03,1.622700,26.150515,1.586585,10.934123
min,3.597122e-03,3.623188e-03,-8.118941,0.059976,-8.108524,0.060193
25%,6.120209e+02,1.351406e+02,9.257437,24.739057,7.078318,11.625000
50%,1.194627e+03,2.595071e+02,10.222345,34.563379,8.019630,16.109224
75%,2.221939e+03,4.750045e+02,11.117603,47.137446,8.891797,21.794599
max,6.097267e+07,4.191372e+06,25.861659,7808.500000,21.998991,2047.284055


In [ ]:
pickle.dump(final_pop_2019, open('/content/drive/MyDrive/Scripts_and_Data_Transfer/3.Arch_2_Procedure/5.Train_Regression_and_Classification_Models/Inputs/population_data/population_2019_raw.pickle', 'wb'))

In [ ]:
X= final_pop_2019.copy().to_numpy().astype(float)

In [ ]:
for i in range(0,6):
  print(i)
  X[:,i] = StandardScaler().fit_transform(X[:,i].reshape(-1,1)).reshape(-1)

0
1
2
3
4
5


In [ ]:
X = pd.DataFrame(X, index=final_pop_2019.index.copy(), columns = final_pop_2019.columns)

In [ ]:
X

,pop,hh,log_pop,sqrt_pop,log_hh,sqrt_hh
Village_ID,,,,,,
27378,-0.010717,-0.018753,-0.127615,-0.277275,0.109255,-0.116993
27379,-0.011752,-0.022067,-0.241814,-0.350502,0.006275,-0.199610
27380,0.007654,0.026277,0.889576,0.631396,0.926002,0.733766
27381,-0.008599,-0.015274,0.068401,-0.140084,0.206143,-0.034874
27382,0.078694,0.189885,2.037109,2.522566,1.970338,2.558015
...,...,...,...,...,...,...
644661,-0.008240,0.005013,0.097715,-0.118236,0.627428,0.377737
644682,0.269315,1.026046,2.997459,5.370732,3.336000,7.230545
644683,0.041159,0.239585,1.607758,1.669413,2.141746,2.970728


In [ ]:
X.describe()

,pop,hh,log_pop,sqrt_pop,log_hh,sqrt_hh
count,3.159090e+05,3.159090e+05,3.159090e+05,3.159090e+05,3.159090e+05,3.159090e+05
mean,-1.012801e-17,9.925468e-17,-4.672440e-16,1.508858e-15,-7.133256e-16,1.777099e-15
std,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00
min,-1.930251e-02,-4.969750e-02,-1.122467e+01,-1.452169e+00,-1.008961e+01,-1.612197e+00
25%,-1.375757e-02,-3.416639e-02,-5.163392e-01,-5.084358e-01,-5.175616e-01,-5.545146e-01
50%,-8.479102e-03,-1.987313e-02,7.829249e-02,-1.327515e-01,7.573430e-02,-1.444011e-01
75%,8.284405e-04,4.893683e-03,6.300024e-01,3.480836e-01,6.254488e-01,3.755659e-01
max,5.523990e+02,4.816588e+02,9.716140e+00,2.971443e+02,8.886726e+00,1.856207e+02


In [ ]:
pickle.dump(X, open('/content/drive/MyDrive/Scripts_and_Data_Transfer/3.Arch_2_Procedure/5.Train_Regression_and_Classification_Models/Inputs/population_data/population_2019.pickle', 'wb'))

#Verify

In [ ]:
p01 = pickle.load(open('/content/drive/MyDrive/Scripts_and_Data_Transfer/3.Arch_2_Procedure/5.Train_Regression_and_Classification_Models/Inputs/population_data/population_2001.pickle', 'rb'))
p01

,pop,hh,log_pop,sqrt_pop,log_hh,sqrt_hh
Village_ID,,,,,,
27378,-0.349976,-0.409368,-0.024231,-0.263761,-0.164901,-0.383259
27379,-0.375081,-0.420606,-0.069124,-0.305193,-0.189052,-0.403686
27380,0.583077,0.624547,0.918069,0.881430,0.969909,0.940236
27381,1.217665,0.912995,1.253267,1.449883,1.138360,1.213464
27382,-0.562668,-0.592926,-0.497785,-0.654403,-0.666802,-0.758511
...,...,...,...,...,...,...
644661,0.067736,0.654516,0.514551,0.319716,0.988874,0.969796
644682,0.548210,1.321316,0.895639,0.846915,1.334884,1.564547
644683,0.781124,1.718400,1.036150,1.070134,1.493395,1.875532


In [ ]:
p11 = pickle.load(open('/content/drive/MyDrive/Scripts_and_Data_Transfer/3.Arch_2_Procedure/5.Train_Regression_and_Classification_Models/Inputs/population_data/population_2011.pickle', 'rb'))
p11

,pop,hh,log_pop,sqrt_pop,log_hh,sqrt_hh
Village_ID,,,,,,
27378,-0.362916,-0.323749,-0.039997,-0.286324,0.007857,-0.234668
27379,-0.405461,-0.359134,-0.119441,-0.359097,-0.056165,-0.294303
27380,0.612072,0.655232,0.929300,0.916267,0.975670,0.971300
27381,0.248667,0.257152,0.675470,0.536375,0.692629,0.551118
27382,0.453119,0.274845,0.826955,0.756620,0.707235,0.571260
...,...,...,...,...,...,...
644661,-0.103511,0.439974,0.333939,0.102866,0.833448,0.752116
644682,3.913440,6.024882,1.985116,3.220821,2.376310,4.328670
644683,1.492515,3.026016,1.349680,1.676138,1.847751,2.756499


In [ ]:
p19 = pickle.load(open('/content/drive/MyDrive/Scripts_and_Data_Transfer/3.Arch_2_Procedure/5.Train_Regression_and_Classification_Models/Inputs/population_data/population_2019.pickle', 'rb'))
p19

,pop,hh,log_pop,sqrt_pop,log_hh,sqrt_hh
Village_ID,,,,,,
27378,-0.010717,-0.018753,-0.127615,-0.277275,0.109255,-0.116993
27379,-0.011752,-0.022067,-0.241814,-0.350502,0.006275,-0.199610
27380,0.007654,0.026277,0.889576,0.631396,0.926002,0.733766
27381,-0.008599,-0.015274,0.068401,-0.140084,0.206143,-0.034874
27382,0.078694,0.189885,2.037109,2.522566,1.970338,2.558015
...,...,...,...,...,...,...
644661,-0.008240,0.005013,0.097715,-0.118236,0.627428,0.377737
644682,0.269315,1.026046,2.997459,5.370732,3.336000,7.230545
644683,0.041159,0.239585,1.607758,1.669413,2.141746,2.970728


#Add population to all Data Frames

In [ ]:
IN = pickle.load(open('/content/drive/MyDrive/Scripts_and_Data_Transfer/4.Training_CNN_(Arch_3)/Data/2nd_Level_Regression_Inp/household_input_2001_combined_arch3.pickle', 'rb'))
# IN = pickle.load(open('/content/drive/MyDrive/Scripts_and_Data_Transfer/3.Arch_2_Procedure/5.Train_Regression_and_Classification_Models/Inputs/Household_Indicators_Input/household_input_2017_combined_new.pickle', 'rb'))
IN

,BF_0_OUT_1,BF_0_OUT_2,BF_0_OUT_3,FC_0_OUT_1,FC_0_OUT_2,FC_0_OUT_3,MSW_0_OUT_1,MSW_0_OUT_2,MSW_0_OUT_3,BF_1_OUT_1,...,dc_score_sqrt_intensity,dc_score_none_size_only,dc_score_none_size_intensity_only,dc_score_none_intensity_only,dc_score_log_size_only,dc_score_log_size_intensity_only,dc_score_log_intensity_only,dc_score_sqrt_size_only,dc_score_sqrt_size_intensity_only,dc_score_sqrt_intensity_only
27378,0.546753,0.227011,0.226236,0.559013,0.221311,0.219676,0.420802,0.333921,0.245276,0.361129,...,-0.824553,1.217479,0.729519,-1.083625,0.950924,0.616500,-1.010814,1.124290,0.692364,-1.122401
27379,0.499389,0.244926,0.255685,0.536593,0.236775,0.226632,0.421868,0.311766,0.266366,0.476923,...,-0.777987,1.217479,0.729519,-1.083625,0.950924,0.616500,-1.010814,1.124290,0.692364,-1.122401
27380,0.512762,0.262645,0.224593,0.558707,0.222080,0.219213,0.434855,0.308120,0.257025,0.361129,...,-0.800027,1.217479,0.729519,-1.083625,0.950924,0.616500,-1.010814,1.124290,0.692364,-1.122401
27381,0.361129,0.392448,0.246423,0.540473,0.228022,0.231505,0.453437,0.295164,0.251400,0.476923,...,-0.751708,1.217479,0.729519,-1.083625,0.950924,0.616500,-1.010814,1.124290,0.692364,-1.122401
27382,0.476923,0.264620,0.258457,0.546135,0.226884,0.226981,0.473598,0.276481,0.249921,0.361129,...,-0.700804,1.217479,0.729519,-1.083625,0.950924,0.616500,-1.010814,1.124290,0.692364,-1.122401
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
644661,0.449178,0.260259,0.290563,0.455540,0.263809,0.280652,0.241466,0.307173,0.451361,0.546714,...,0.345057,0.556557,-0.365403,-1.269102,0.705079,0.090211,-1.448932,0.672424,-0.137623,-1.431296
644682,0.465442,0.254541,0.280016,0.453913,0.248156,0.297931,0.237646,0.260535,0.501820,0.497353,...,2.983548,0.556557,-0.365403,-1.269102,0.705079,0.090211,-1.448932,0.672424,-0.137623,-1.431296
644683,0.507918,0.236689,0.255393,0.432863,0.259640,0.307497,0.224760,0.324476,0.450764,0.444778,...,1.750626,0.556557,-0.365403,-1.269102,0.705079,0.090211,-1.448932,0.672424,-0.137623,-1.431296
644687,0.444778,0.267361,0.287861,0.505766,0.241835,0.252399,0.277376,0.401206,0.321419,0.449178,...,0.496703,0.556557,-0.365403,-1.269102,0.705079,0.090211,-1.448932,0.672424,-0.137623,-1.431296


In [ ]:
IN.drop(['pop', 'hh', 'log_pop', 'sqrt_pop', 'log_hh', 'sqrt_hh'], axis=1, inplace=True)
IN

,BF_0_OUT_1,BF_0_OUT_2,BF_0_OUT_3,FC_0_OUT_1,FC_0_OUT_2,FC_0_OUT_3,MSW_0_OUT_1,MSW_0_OUT_2,MSW_0_OUT_3,BF_1_OUT_1,BF_1_OUT_2,BF_1_OUT_3,FC_1_OUT_1,FC_1_OUT_2,FC_1_OUT_3,MSW_1_OUT_1,MSW_1_OUT_2,MSW_1_OUT_3,BF_2_OUT_1,BF_2_OUT_2,BF_2_OUT_3,FC_2_OUT_1,FC_2_OUT_2,FC_2_OUT_3,MSW_2_OUT_1,MSW_2_OUT_2,MSW_2_OUT_3,BF_3_OUT_1,BF_3_OUT_2,BF_3_OUT_3,FC_3_OUT_1,FC_3_OUT_2,FC_3_OUT_3,MSW_3_OUT_1,MSW_3_OUT_2,MSW_3_OUT_3,BF_4_OUT_1,BF_4_OUT_2,BF_4_OUT_3,FC_4_OUT_1,...,ASSET_2_OUT_3,ASSET_3_OUT_3,ASSET_4_OUT_3,ASSET_5_OUT_3,ASSET_N_OUT_3,ASSET_0_OUT_4,ASSET_1_OUT_4,ASSET_2_OUT_4,ASSET_3_OUT_4,ASSET_4_OUT_4,ASSET_5_OUT_4,ASSET_N_OUT_4,LIT_0_OUT_1,LIT_1_OUT_1,LIT_2_OUT_1,LIT_3_OUT_1,LIT_4_OUT_1,LIT_5_OUT_1,LIT_N_OUT_1,dc_score_none_none,dc_score_none_size,dc_score_none_size_intensity,dc_score_none_intensity,dc_score_log_none,dc_score_log_size,dc_score_log_size_intensity,dc_score_log_intensity,dc_score_sqrt_none,dc_score_sqrt_size,dc_score_sqrt_size_intensity,dc_score_sqrt_intensity,dc_score_none_size_only,dc_score_none_size_intensity_only,dc_score_none_intensity_only,dc_score_log_size_only,dc_score_log_size_intensity_only,dc_score_log_intensity_only,dc_score_sqrt_size_only,dc_score_sqrt_size_intensity_only,dc_score_sqrt_intensity_only
27378,0.450654,0.252962,0.296384,0.503688,0.228672,0.267641,0.283980,0.416766,0.299254,0.404751,0.341285,0.253963,0.516307,0.223787,0.259906,0.302083,0.329116,0.368801,0.433088,0.307435,0.259478,0.485192,0.219216,0.295592,0.276847,0.337818,0.385335,0.411186,0.296979,0.291835,0.514276,0.228837,0.256888,0.321716,0.249432,0.428852,0.353726,0.320924,0.325350,0.462787,...,2.755311,1.483604,1.352645,2.738275,2.478230,1.171848,0.869074,0.891126,0.797312,0.526659,1.131836,0.933604,1.123079,1.375410,1.281753,1.010488,1.402736,0.961566,1.490337,-0.010794,-0.235655,-0.118069,0.997633,0.405644,-0.065347,0.220358,1.767178,0.178014,-0.201343,0.025999,1.726904,0.971891,0.586182,2.305343,0.751182,0.483776,1.696890,0.889434,0.550484,2.077334
27379,0.455719,0.267446,0.276835,0.521914,0.222991,0.255094,0.273379,0.385237,0.341383,0.411186,0.296979,0.291835,0.514276,0.228837,0.256888,0.321716,0.249432,0.428852,0.373860,0.334283,0.291856,0.478079,0.225658,0.296263,0.285217,0.270897,0.443887,0.429027,0.269969,0.301004,0.478039,0.229054,0.292907,0.310845,0.287189,0.401965,0.496368,0.242131,0.261501,0.366809,...,2.665363,2.737349,2.823045,2.900228,2.738685,1.390445,0.802736,0.669899,1.128989,0.833902,0.823613,0.943285,1.723978,1.012821,1.591440,0.964275,0.578266,0.493267,1.144557,-0.073575,-0.235961,-0.118457,1.108978,0.199958,-0.066059,0.219650,1.861324,0.007298,-0.201895,0.025414,1.863071,0.971891,0.586182,2.305343,0.751182,0.483776,1.696890,0.889434,0.550484,2.077334
27380,0.433088,0.307435,0.259478,0.485192,0.219216,0.295592,0.276847,0.337818,0.385335,0.404751,0.341285,0.253963,0.516307,0.223787,0.259906,0.302083,0.329116,0.368801,0.450654,0.252962,0.296384,0.503688,0.228672,0.267641,0.283980,0.416766,0.299254,0.411186,0.296979,0.291835,0.514276,0.228837,0.256888,0.321716,0.249432,0.428852,0.353726,0.320924,0.325350,0.462787,...,1.760514,1.483604,1.352645,2.738275,2.262621,0.892362,0.869074,1.164270,0.797312,0.526659,1.131836,0.994104,1.422714,1.375410,1.019074,1.010488,1.402736,0.961566,1.425692,-0.047625,-0.231038,-0.112210,0.966522,0.288532,-0.054641,0.231014,1.739732,0.079335,-0.193022,0.034803,1.688040,0.971891,0.586182,2.305343,0.751182,0.483776,1.696890,0.889434,0.550484,2.077334
27381,0.404751,0.341285,0.253963,0.516307,0.223787,0.259906,0.302083,0.329116,0.368801,0.411186,0.296979,0.291835,0.514276,0.228837,0.256888,0.321716,0.249432,0.428852,0.450654,0.252962,0.296384,0.503688,0.228672,0.267641,0.283980,0.416766,0.299254,0.433088,0.307435,0.259478,0.485192,0.219216,0.295592,0.276847,0.337818,0.385335,0.353726,0.320924,0.325350,0.462787,...,1.760514,2.748583,1.352645,2.738275,2.190619,0.865478,0.802736,1.164270,0.889716,0.526659,1.131836,0.999923,1.526776,1.012821,1.019074,1.278734,1.402736,0.961566,1.403241,-0.091262,-0.23313

In [ ]:
final_pop_toAdd = pickle.load(open('/content/drive/MyDrive/Scripts_and_Data_Transfer/3.Arch_2_Procedure/5.Train_Regression_and_Classification_Models/Inputs/population_data/population_2001.pickle', 'rb'))
final_pop_toAdd

,pop,hh,log_pop,sqrt_pop,log_hh,sqrt_hh
Village_ID,,,,,,
27378,-0.349976,-0.409368,-0.024231,-0.263761,-0.164901,-0.383259
27379,-0.375081,-0.420606,-0.069124,-0.305193,-0.189052,-0.403686
27380,0.583077,0.624547,0.918069,0.881430,0.969909,0.940236
27381,1.217665,0.912995,1.253267,1.449883,1.138360,1.213464
27382,-0.562668,-0.592926,-0.497785,-0.654403,-0.666802,-0.758511
...,...,...,...,...,...,...
644661,0.067736,0.654516,0.514551,0.319716,0.988874,0.969796
644682,0.548210,1.321316,0.895639,0.846915,1.334884,1.564547
644683,0.781124,1.718400,1.036150,1.070134,1.493395,1.875532


In [ ]:
IN = IN.join(final_pop_toAdd, how = 'inner')
IN

,BF_0_OUT_1,BF_0_OUT_2,BF_0_OUT_3,FC_0_OUT_1,FC_0_OUT_2,FC_0_OUT_3,MSW_0_OUT_1,MSW_0_OUT_2,MSW_0_OUT_3,BF_1_OUT_1,...,dc_score_log_intensity_only,dc_score_sqrt_size_only,dc_score_sqrt_size_intensity_only,dc_score_sqrt_intensity_only,pop,hh,log_pop,sqrt_pop,log_hh,sqrt_hh
27378,0.546753,0.227011,0.226236,0.559013,0.221311,0.219676,0.420802,0.333921,0.245276,0.361129,...,-1.010814,1.124290,0.692364,-1.122401,-0.349976,-0.409368,-0.024231,-0.263761,-0.164901,-0.383259
27379,0.499389,0.244926,0.255685,0.536593,0.236775,0.226632,0.421868,0.311766,0.266366,0.476923,...,-1.010814,1.124290,0.692364,-1.122401,-0.375081,-0.420606,-0.069124,-0.305193,-0.189052,-0.403686
27380,0.512762,0.262645,0.224593,0.558707,0.222080,0.219213,0.434855,0.308120,0.257025,0.361129,...,-1.010814,1.124290,0.692364,-1.122401,0.583077,0.624547,0.918069,0.881430,0.969909,0.940236
27381,0.361129,0.392448,0.246423,0.540473,0.228022,0.231505,0.453437,0.295164,0.251400,0.476923,...,-1.010814,1.124290,0.692364,-1.122401,1.217665,0.912995,1.253267,1.449883,1.138360,1.213464
27382,0.476923,0.264620,0.258457,0.546135,0.226884,0.226981,0.473598,0.276481,0.249921,0.361129,...,-1.010814,1.124290,0.692364,-1.122401,-0.562668,-0.592926,-0.497785,-0.654403,-0.666802,-0.758511
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
644661,0.449178,0.260259,0.290563,0.455540,0.263809,0.280652,0.241466,0.307173,0.451361,0.546714,...,-1.448932,0.672424,-0.137623,-1.431296,0.067736,0.654516,0.514551,0.319716,0.988874,0.969796
644682,0.465442,0.254541,0.280016,0.453913,0.248156,0.297931,0.237646,0.260535,0.501820,0.497353,...,-1.448932,0.672424,-0.137623,-1.431296,0.548210,1.321316,0.895639,0.846915,1.334884,1.564547
644683,0.507918,0.236689,0.255393,0.432863,0.259640,0.307497,0.224760,0.324476,0.450764,0.444778,...,-1.448932,0.672424,-0.137623,-1.431296,0.781124,1.718400,1.036150,1.070134,1.493395,1.875532
644687,0.444778,0.267361,0.287861,0.505766,0.241835,0.252399,0.277376,0.401206,0.321419,0.449178,...,-1.448932,0.672424,-0.137623,-1.431296,1.345977,2.819744,1.308530,1.553807,1.830347,2.629876


In [ ]:
IN.describe()

,BF_0_OUT_1,BF_0_OUT_2,BF_0_OUT_3,FC_0_OUT_1,FC_0_OUT_2,FC_0_OUT_3,MSW_0_OUT_1,MSW_0_OUT_2,MSW_0_OUT_3,BF_1_OUT_1,...,dc_score_log_intensity_only,dc_score_sqrt_size_only,dc_score_sqrt_size_intensity_only,dc_score_sqrt_intensity_only,pop,hh,log_pop,sqrt_pop,log_hh,sqrt_hh
count,311831.000000,311831.000000,311831.000000,311831.000000,311831.000000,311831.000000,311831.000000,311831.000000,311831.000000,311831.000000,...,311831.000000,311831.000000,311831.000000,311831.000000,311831.000000,311831.000000,311831.000000,311831.000000,311831.000000,311831.000000
mean,0.511049,0.240056,0.248895,0.468188,0.285135,0.246677,0.295501,0.393217,0.311282,0.510048,...,0.004089,-0.003368,-0.018014,0.001989,-0.002787,-0.002118,-0.002255,-0.002998,-0.001712,-0.002352
std,0.045314,0.031728,0.022732,0.068245,0.058132,0.025279,0.052395,0.067455,0.060219,0.047547,...,0.995397,1.001095,1.011550,0.985289,0.998319,0.999324,0.998502,0.998090,0.998529,0.998528
min,0.210320,0.196067,0.181938,0.220638,0.207725,0.166022,0.186892,0.178646,0.194237,0.228269,...,-3.947089,-1.955757,-2.896021,-2.490176,-0.883448,-0.851404,-6.217725,-1.943004,-4.685292,-1.845927
25%,0.503121,0.224582,0.234281,0.419212,0.237680,0.229835,0.252447,0.341286,0.264922,0.502540,...,-0.479359,-0.974852,-0.836591,-0.673871,-0.574523,-0.555465,-0.532826,-0.679557,-0.540500,-0.673238
50%,0.524596,0.229905,0.242839,0.484815,0.262915,0.241318,0.287827,0.397256,0.294933,0.524448,...,0.311135,-0.068738,0.692364,0.179735,-0.288609,-0.285748,0.077187,-0.166408,0.066606,-0.173629
75%,0.539047,0.241462,0.255225,0.527039,0.323140,0.254477,0.326838,0.446802,0.345244,0.539262,...,0.746976,1.124290,0.692364,0.768741,0.210693,0.190003,0.645040,0.488247,0.641057,0.472105
max,0.574449,0.607743,0.445224,0.576197,0.581572,0.428554,0.550440,0.559699,0.587284,0.572482,...,1.745979,1.124290,0.692364,2.537271,38.481221,41.816333,3.985834,12.949974,4.149606,13.761138


In [ ]:
pickle.dump(IN, open('/content/drive/MyDrive/Scripts_and_Data_Transfer/3.Arch_2_Procedure/5.Train_Regression_and_Classification_Models/Inputs/Household_Indicators_Input/Arch3_household_input_2001_combined_new.pickle', 'wb'))

#Rough work

In [ ]:
IN = pickle.load(open('/content/drive/MyDrive/Scripts_and_Data_Transfer/3.Arch_2_Procedure/5.Train_Regression_and_Classification_Models/Inputs/Household_Indicators_Input/household_input_2011_combined_new.pickle', 'rb'))
IN

,BF_0_OUT_1,BF_0_OUT_2,BF_0_OUT_3,FC_0_OUT_1,FC_0_OUT_2,FC_0_OUT_3,MSW_0_OUT_1,MSW_0_OUT_2,MSW_0_OUT_3,ASSET_0_OUT_1,...,dc_score_log_intensity_only,dc_score_sqrt_size_only,dc_score_sqrt_size_intensity_only,dc_score_sqrt_intensity_only,pop,hh,log_pop,sqrt_pop,log_hh,sqrt_hh
27378,0.053910,0.939236,0.006854,0.809554,0.009099,0.181347,0.059269,0.030144,0.910587,0.000302,...,1.696890,0.889434,0.550484,2.077334,-0.362916,-0.323749,-0.039997,-0.286324,0.007857,-0.234668
27379,0.020868,0.978828,0.000304,0.777852,0.015734,0.206415,0.152171,0.304250,0.543579,0.014037,...,1.696890,0.889434,0.550484,2.077334,-0.405461,-0.359134,-0.119441,-0.359097,-0.056165,-0.294303
27380,0.162426,0.836869,0.000704,0.343328,0.007851,0.648821,0.043962,0.010956,0.945082,0.000115,...,1.696890,0.889434,0.550484,2.077334,0.612072,0.655232,0.929300,0.916267,0.975670,0.971300
27381,0.047692,0.879542,0.072766,0.737420,0.037230,0.225349,0.422232,0.012824,0.564944,0.000402,...,1.696890,0.889434,0.550484,2.077334,0.248667,0.257152,0.675470,0.536375,0.692629,0.551118
27382,0.764473,0.017324,0.218203,0.879136,0.005643,0.115220,0.040126,0.006185,0.953689,0.000030,...,1.696890,0.889434,0.550484,2.077334,0.453119,0.274845,0.826955,0.756620,0.707235,0.571260
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
644661,0.003038,0.996960,0.000002,0.001660,0.000005,0.998335,0.032586,0.044351,0.923062,0.000006,...,-0.937417,0.363136,-0.009947,-0.916022,-0.103511,0.439974,0.333939,0.102866,0.833448,0.752116
644682,0.020880,0.000908,0.978212,0.000301,0.000032,0.999667,0.000263,0.029987,0.969750,0.000035,...,-0.937417,0.363136,-0.009947,-0.916022,3.913440,6.024882,1.985116,3.220821,2.376310,4.328670
644683,0.015891,0.014879,0.969230,0.144198,0.000117,0.855685,0.051282,0.038199,0.910518,0.016459,...,-0.937417,0.363136,-0.009947,-0.916022,1.492515,3.026016,1.349680,1.676138,1.847751,2.756499
644687,0.995348,0.000146,0.004507,0.023803,0.000146,0.976051,0.108068,0.031764,0.860168,0.000855,...,-0.937417,0.363136,-0.009947,-0.916022,1.118474,2.539475,1.194333,1.374471,1.724051,2.449161


In [ ]:
print(list(IN.columns))

['BF_0_OUT_1', 'BF_0_OUT_2', 'BF_0_OUT_3', 'FC_0_OUT_1', 'FC_0_OUT_2', 'FC_0_OUT_3', 'MSW_0_OUT_1', 'MSW_0_OUT_2', 'MSW_0_OUT_3', 'ASSET_0_OUT_1', 'ASSET_0_OUT_2', 'ASSET_0_OUT_3', 'LIT_0_OUT_1', 'LIT_0_OUT_2', 'LIT_0_OUT_3', 'BF_1_OUT_1', 'BF_1_OUT_2', 'BF_1_OUT_3', 'FC_1_OUT_1', 'FC_1_OUT_2', 'FC_1_OUT_3', 'MSW_1_OUT_1', 'MSW_1_OUT_2', 'MSW_1_OUT_3', 'ASSET_1_OUT_1', 'ASSET_1_OUT_2', 'ASSET_1_OUT_3', 'LIT_1_OUT_1', 'LIT_1_OUT_2', 'LIT_1_OUT_3', 'BF_2_OUT_1', 'BF_2_OUT_2', 'BF_2_OUT_3', 'FC_2_OUT_1', 'FC_2_OUT_2', 'FC_2_OUT_3', 'MSW_2_OUT_1', 'MSW_2_OUT_2', 'MSW_2_OUT_3', 'ASSET_2_OUT_1', 'ASSET_2_OUT_2', 'ASSET_2_OUT_3', 'LIT_2_OUT_1', 'LIT_2_OUT_2', 'LIT_2_OUT_3', 'BF_3_OUT_1', 'BF_3_OUT_2', 'BF_3_OUT_3', 'FC_3_OUT_1', 'FC_3_OUT_2', 'FC_3_OUT_3', 'MSW_3_OUT_1', 'MSW_3_OUT_2', 'MSW_3_OUT_3', 'ASSET_3_OUT_1', 'ASSET_3_OUT_2', 'ASSET_3_OUT_3', 'LIT_3_OUT_1', 'LIT_3_OUT_2', 'LIT_3_OUT_3', 'BF_4_OUT_1', 'BF_4_OUT_2', 'BF_4_OUT_3', 'FC_4_OUT_1', 'FC_4_OUT_2', 'FC_4_OUT_3', 'MSW_4_OUT_1', 

In [ ]:
IN.columns[:127]

Index(['BF_0_OUT_1', 'BF_0_OUT_2', 'BF_0_OUT_3', 'FC_0_OUT_1', 'FC_0_OUT_2',
       'FC_0_OUT_3', 'MSW_0_OUT_1', 'MSW_0_OUT_2', 'MSW_0_OUT_3',
       'ASSET_0_OUT_1',
       ...
       'dc_score_none_size_only', 'dc_score_none_size_intensity_only',
       'dc_score_none_intensity_only', 'dc_score_log_size_only',
       'dc_score_log_size_intensity_only', 'dc_score_log_intensity_only',
       'dc_score_sqrt_size_only', 'dc_score_sqrt_size_intensity_only',
       'dc_score_sqrt_intensity_only', 'pop'],
      dtype='object', length=127)